# Data PreProcessing

In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import datetime as dt

In [2]:
# load data
df = pd.read_parquet(
    "/mnt/E/Repos/mlops/dataops/data/raw/yellow_tripdata_2025-01.parquet",
    engine="pyarrow",
)

In [3]:
## Check column Data Types and counts

print(df.info(show_counts=True))

<class 'pandas.DataFrame'>
RangeIndex: 3475226 entries, 0 to 3475225
Data columns (total 20 columns):
 #   Column                 Non-Null Count    Dtype         
---  ------                 --------------    -----         
 0   VendorID               3475226 non-null  int32         
 1   tpep_pickup_datetime   3475226 non-null  datetime64[us]
 2   tpep_dropoff_datetime  3475226 non-null  datetime64[us]
 3   passenger_count        2935077 non-null  float64       
 4   trip_distance          3475226 non-null  float64       
 5   RatecodeID             2935077 non-null  float64       
 6   store_and_fwd_flag     2935077 non-null  str           
 7   PULocationID           3475226 non-null  int32         
 8   DOLocationID           3475226 non-null  int32         
 9   payment_type           3475226 non-null  int64         
 10  fare_amount            3475226 non-null  float64       
 11  extra                  3475226 non-null  float64       
 12  mta_tax                3475226 non-null

In [4]:
# creating initial preprocessed data frame as copy or original raw data with only required columns

df_preprocessed = df[
    [
        "PULocationID",
        "DOLocationID",
        "trip_distance",
        "tpep_pickup_datetime",
        "total_amount",
    ]
].copy()

In [5]:
df_preprocessed["pickup_hour"] = df_preprocessed["tpep_pickup_datetime"].dt.hour
df_preprocessed["pickup_day_of_week"] = df_preprocessed[
    "tpep_pickup_datetime"
].dt.dayofweek
df_preprocessed["pickup_month"] = df_preprocessed["tpep_pickup_datetime"].dt.month
df_preprocessed = df_preprocessed.drop(columns=["tpep_pickup_datetime"])

In [6]:
print(df_preprocessed.info(show_counts=True))

print(df_preprocessed.corr()["total_amount"])
print(df_preprocessed.info(show_counts=True))

<class 'pandas.DataFrame'>
RangeIndex: 3475226 entries, 0 to 3475225
Data columns (total 7 columns):
 #   Column              Non-Null Count    Dtype  
---  ------              --------------    -----  
 0   PULocationID        3475226 non-null  int32  
 1   DOLocationID        3475226 non-null  int32  
 2   trip_distance       3475226 non-null  float64
 3   total_amount        3475226 non-null  float64
 4   pickup_hour         3475226 non-null  int32  
 5   pickup_day_of_week  3475226 non-null  int32  
 6   pickup_month        3475226 non-null  int32  
dtypes: float64(2), int32(5)
memory usage: 119.3 MB
None
PULocationID         -0.004792
DOLocationID         -0.004276
trip_distance         0.000201
total_amount          1.000000
pickup_hour           0.000332
pickup_day_of_week   -0.001803
pickup_month          0.000013
Name: total_amount, dtype: float64
<class 'pandas.DataFrame'>
RangeIndex: 3475226 entries, 0 to 3475225
Data columns (total 7 columns):
 #   Column              Non-N

In [7]:
# Fix The "Broken" Feature: trip_distance

# 1. Filter out impossible distances and amounts
df_preprocessed = df_preprocessed[
    (df_preprocessed["trip_distance"] > 0)
    & (df_preprocessed["trip_distance"] < 100)
    & (df_preprocessed["total_amount"] > 2.5)  # Minimum base fare
    & (df_preprocessed["total_amount"] < 500)
].copy()

In [8]:
# 3. Check correlation again
print(df_preprocessed.corr()["total_amount"])
print(df_preprocessed.info(show_counts=True))

PULocationID         -0.116731
DOLocationID         -0.080865
trip_distance         0.911123
total_amount          1.000000
pickup_hour           0.014509
pickup_day_of_week   -0.022904
pickup_month          0.000197
Name: total_amount, dtype: float64
<class 'pandas.DataFrame'>
Index: 3308750 entries, 0 to 3475225
Data columns (total 7 columns):
 #   Column              Non-Null Count    Dtype  
---  ------              --------------    -----  
 0   PULocationID        3308750 non-null  int32  
 1   DOLocationID        3308750 non-null  int32  
 2   trip_distance       3308750 non-null  float64
 3   total_amount        3308750 non-null  float64
 4   pickup_hour         3308750 non-null  int32  
 5   pickup_day_of_week  3308750 non-null  int32  
 6   pickup_month        3308750 non-null  int32  
dtypes: float64(2), int32(5)
memory usage: 138.8 MB
None


In [10]:
# save data to file
df_preprocessed.to_parquet(
    "/mnt/E/Repos/mlops/dataops/data/preprocessed/cleaned_yellow_tripdata_2025-01.parquet"
)